## Contents

- Keep the mean for each feature by week
- Imputation - keep the columns which have more than 50% of the values
- Fill the missing values with KNN imputation
    

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.impute import KNNImputer
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE

pd.set_option('display.max_columns', None)

In [ ]:
KEEP_ONLY_MALES = True

In [ ]:
# read pickle file from cleaning process
biometrics = pd.read_pickle('../data_processed/filtered_biometrics.pkl')
biometrics.drop(columns=['MeasureProvidedBy'], inplace=True)
biometrics.shape

(4555157, 11)

### Pivot table to put features as columns

In [ ]:
# Create dummy variables for the gender column
biometrics = pd.get_dummies(biometrics, columns=['Gender'], prefix='', prefix_sep='')

# Rename the dummy columns to gender_m and gender_f
biometrics.rename(columns={'M': 'gender_m', 'F': 'gender_f'}, inplace=True)

biometrics['gender_m'] = biometrics['gender_m'].astype(int)
biometrics['gender_f'] = biometrics['gender_f'].astype(int)

# Pivot the dataframe to make each value of 'BiometricName' a new column
biometrics_pivot = biometrics.pivot_table(index=['CloudId', 'gender_m', 'gender_f', 'Age', 'MeasuredOnWeek'], 
                                          columns='BiometricName', 
                                          values='Value', 
                                          aggfunc='last').reset_index()

# Merge the pivoted dataframe with the original dataframe to keep other columns
biometrics_merged = pd.merge(biometrics.drop(columns=['BiometricName', 'Value']), 
                             biometrics_pivot, 
                             on=['CloudId', 'gender_m', 'gender_f', 'Age', 'MeasuredOnWeek'], 
                             how='left').drop_duplicates()

# Group by 'MeasuredOnDate' and aggregate to put measurements on the same row
biometrics_final = biometrics_merged.groupby(['CloudId', 'gender_m', 'gender_f', 'Age', 'MeasuredOnDate']).first().reset_index()
biometrics_final.head()

,CloudId,gender_m,gender_f,Age,MeasuredOnDate,MeasuredOnUTC,MeasuredOn,MeasuredOnDay,MeasuredOnWeek,MeasuredOnYear,BFM Control,BMI,Basal Metabolic Rate,Basal Metabolic Rate Score,Body cell mass,Bone Mass,Bone Mineral Content,Chest Circumference,Degree Of Obesity Perc,Diastolic Blood Pressure,Dry Lean Mass,ECW/TBW,ECW/TBW-LA,ECW/TBW-LLL,ECW/TBW-RA,ECW/TBW-RL,ECW/TBW-TR,Extra Cellular Water,Extra Cellular Water Perc,FFM Control,Fat Free Mass,Fat Free Mass Perc of Ideal Left Arm,Fat Free Mass Perc of Ideal Left Leg,Fat Free Mass Perc of Ideal Right Arm,Fat Free Mass Perc of Ideal Right Leg,Fat Free Mass Perc of Ideal Trunk,Fat Mass,Fat Mass Perc of Ideal Left Arm,Fat Mass Perc of Ideal Left Leg,Fat Mass Perc of Ideal Right Arm,Fat Mass Perc of Ideal Right Leg,Fat Mass Perc of Ideal Trunk,Fat mass Perc,Growth Score,HR At Rest,Height,Hip Circumference,InBody Score,Intra Cellular Water,Left Arm Circumference,Left Arm Fat Free Mass,Left Arm Fat Mass,Left Arm Fat Perc,Left Arm Fat Perc Score,Left Arm Muscle Circumference,Left Arm Muscle Mass,Left Arm Muscle Mass Score,Left Leg Fat Free Mass,Left Leg Fat Mass,Left Leg Fat Perc,Left Leg Fat Perc Score,Left Leg Muscle Mass,Left Leg Muscle Mass Score,Left Thigh Circumference,Leg Muscle Score,Metabolic Age,Minerals,Muscle Mass,Muscle Mass Balance Arm,Muscle Mass Balance Leg,Muscle Score,Neck Circumference,Obesity Degree,Obesity Degree of a Child,Power Threshold,Protein,Right Arm Circumference,Right Arm Fat Free Mass,Right Arm Fat Mass,Right Arm Fat Perc,Right Arm Fat Perc Score,Right Arm Muscle Mass,Right Arm Muscle Mass Score,Right Leg Fat Free Mass,Right Leg Fat Mass,Right Leg Fat Perc,Right Leg Fat Perc Score,Right Leg Muscle Mass,Right Leg Muscle Mass Score,Right Thigh Circumference,Segmental ECW LA,Segmental ECW LL,Segmental ECW RA,Segmental ECW RL,Segmental ECW TR,Segmental ICW LA,Segmental ICW LL,Segmental ICW RA,Segmental ICW RL,Segmental ICW TR,Segmental body water LA,Segmental body water LL,Segmental body water RA,Segmental body water RL,Segmental body water TR,Skeletal Muscle Mass,Soft Lean Mass,Standard Body Weight,Standard Fat Perc,Standard Muscle Mass Perc,Systolic Blood Pressure,TBW/FFM,Target Weight,Total Body Water,Total Body Water Perc,Trunk Fat Free Mass,Trunk Fat Mass,Trunk Fat Perc,Trunk Fat Perc Score,Trunk Muscle Mass,Trunk Muscle Mass Score,VFA (Visceral Fat Area),Visceral Fat Rating,Waist Circumference,Waist-Hip ratio,Weight,Weight Control
0,0015d65e3205deb6bb6a8f0d57cc48547918f0f2,1,0,56,2022-03-07,2022-03-07 19:57:09.961000+00:00,2022-03-07 19:57:09+00:00,66,10,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.0,NaN
1,0015d65e3205deb6bb6a8f0d57cc48547918f0f2,1,0,56,2022-05-08,2022-05-08 10:33:57.662000+00:00,2022-05-08 10:33:57+00:00,128,18,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.0,NaN
2,0015d65e3205deb6bb6a8f0d57cc48547918f0f2,1,0,56,2022-06-11,2022-06-11 16:18:22.175000+00:00,2022-06-11 16:18:22+00:00,162,23,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [6]:
# Group by 'CloudId', 'BiometricName', and 'MeasuredOnWeek' and aggregate the values into lists
grouped_measurements = biometrics_final.groupby(['CloudId', 'MeasuredOnWeek']).agg(list).reset_index()
grouped_measurements.head()

,CloudId,MeasuredOnWeek,gender_m,gender_f,Age,MeasuredOnDate,MeasuredOnUTC,MeasuredOn,MeasuredOnDay,MeasuredOnYear,BFM Control,BMI,Basal Metabolic Rate,Basal Metabolic Rate Score,Body cell mass,Bone Mass,Bone Mineral Content,Chest Circumference,Degree Of Obesity Perc,Diastolic Blood Pressure,Dry Lean Mass,ECW/TBW,ECW/TBW-LA,ECW/TBW-LLL,ECW/TBW-RA,ECW/TBW-RL,ECW/TBW-TR,Extra Cellular Water,Extra Cellular Water Perc,FFM Control,Fat Free Mass,Fat Free Mass Perc of Ideal Left Arm,Fat Free Mass Perc of Ideal Left Leg,Fat Free Mass Perc of Ideal Right Arm,Fat Free Mass Perc of Ideal Right Leg,Fat Free Mass Perc of Ideal Trunk,Fat Mass,Fat Mass Perc of Ideal Left Arm,Fat Mass Perc of Ideal Left Leg,Fat Mass Perc of Ideal Right Arm,Fat Mass Perc of Ideal Right Leg,Fat Mass Perc of Ideal Trunk,Fat mass Perc,Growth Score,HR At Rest,Height,Hip Circumference,InBody Score,Intra Cellular Water,Left Arm Circumference,Left Arm Fat Free Mass,Left Arm Fat Mass,Left Arm Fat Perc,Left Arm Fat Perc Score,Left Arm Muscle Circumference,Left Arm Muscle Mass,Left Arm Muscle Mass Score,Left Leg Fat Free Mass,Left Leg Fat Mass,Left Leg Fat Perc,Left Leg Fat Perc Score,Left Leg Muscle Mass,Left Leg Muscle Mass Score,Left Thigh Circumference,Leg Muscle Score,Metabolic Age,Minerals,Muscle Mass,Muscle Mass Balance Arm,Muscle Mass Balance Leg,Muscle Score,Neck Circumference,Obesity Degree,Obesity Degree of a Child,Power Threshold,Protein,Right Arm Circumference,Right Arm Fat Free Mass,Right Arm Fat Mass,Right Arm Fat Perc,Right Arm Fat Perc Score,Right Arm Muscle Mass,Right Arm Muscle Mass Score,Right Leg Fat Free Mass,Right Leg Fat Mass,Right Leg Fat Perc,Right Leg Fat Perc Score,Right Leg Muscle Mass,Right Leg Muscle Mass Score,Right Thigh Circumference,Segmental ECW LA,Segmental ECW LL,Segmental ECW RA,Segmental ECW RL,Segmental ECW TR,Segmental ICW LA,Segmental ICW LL,Segmental ICW RA,Segmental ICW RL,Segmental ICW TR,Segmental body water LA,Segmental body water LL,Segmental body water RA,Segmental body water RL,Segmental body water TR,Skeletal Muscle Mass,Soft Lean Mass,Standard Body Weight,Standard Fat Perc,Standard Muscle Mass Perc,Systolic Blood Pressure,TBW/FFM,Target Weight,Total Body Water,Total Body Water Perc,Trunk Fat Free Mass,Trunk Fat Mass,Trunk Fat Perc,Trunk Fat Perc Score,Trunk Muscle Mass,Trunk Muscle Mass Score,VFA (Visceral Fat Area),Visceral Fat Rating,Waist Circumference,Waist-Hip ratio,Weight,Weight Control
0,0015d65e3205deb6bb6a8f0d57cc48547918f0f2,10,[1],[0],[56],[2022-03-07],[2022-03-07 19:57:09.961000+00:00],[2022-03-07 19:57:09+00:00],[66],[2022],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[88.0],[nan]
1,0015d65e3205deb6bb6a8f0d57cc48547918f0f2,18,[1],[0],[56],[2022-05-08],[2022-05-08 10:33:57.662000+00:00],[2022-05-08 10:33:57+00:00],[128],[2022],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[nan]

### Select mean for each week

In [ ]:
arithmetic_columns = biometrics_final.select_dtypes(include=['float64', 'int64']).columns
# Group by 'CloudId' and 'MeasuredOnWeek' and calculate the mean for the arithmetic columns
mean_values = biometrics_final.groupby(['CloudId', 'MeasuredOnWeek'])[arithmetic_columns].mean().reset_index()
print(mean_values.shape)
mean_values.head()

(90425, 122)


,CloudId,MeasuredOnWeek,gender_m,gender_f,Age,BFM Control,BMI,Basal Metabolic Rate,Basal Metabolic Rate Score,Body cell mass,Bone Mass,Bone Mineral Content,Chest Circumference,Degree Of Obesity Perc,Diastolic Blood Pressure,Dry Lean Mass,ECW/TBW,ECW/TBW-LA,ECW/TBW-LLL,ECW/TBW-RA,ECW/TBW-RL,ECW/TBW-TR,Extra Cellular Water,Extra Cellular Water Perc,FFM Control,Fat Free Mass,Fat Free Mass Perc of Ideal Left Arm,Fat Free Mass Perc of Ideal Left Leg,Fat Free Mass Perc of Ideal Right Arm,Fat Free Mass Perc of Ideal Right Leg,Fat Free Mass Perc of Ideal Trunk,Fat Mass,Fat Mass Perc of Ideal Left Arm,Fat Mass Perc of Ideal Left Leg,Fat Mass Perc of Ideal Right Arm,Fat Mass Perc of Ideal Right Leg,Fat Mass Perc of Ideal Trunk,Fat mass Perc,Growth Score,HR At Rest,Height,Hip Circumference,InBody Score,Intra Cellular Water,Left Arm Circumference,Left Arm Fat Free Mass,Left Arm Fat Mass,Left Arm Fat Perc,Left Arm Fat Perc Score,Left Arm Muscle Circumference,Left Arm Muscle Mass,Left Arm Muscle Mass Score,Left Leg Fat Free Mass,Left Leg Fat Mass,Left Leg Fat Perc,Left Leg Fat Perc Score,Left Leg Muscle Mass,Left Leg Muscle Mass Score,Left Thigh Circumference,Leg Muscle Score,Metabolic Age,Minerals,Muscle Mass,Muscle Mass Balance Arm,Muscle Mass Balance Leg,Muscle Score,Neck Circumference,Obesity Degree,Obesity Degree of a Child,Power Threshold,Protein,Right Arm Circumference,Right Arm Fat Free Mass,Right Arm Fat Mass,Right Arm Fat Perc,Right Arm Fat Perc Score,Right Arm Muscle Mass,Right Arm Muscle Mass Score,Right Leg Fat Free Mass,Right Leg Fat Mass,Right Leg Fat Perc,Right Leg Fat Perc Score,Right Leg Muscle Mass,Right Leg Muscle Mass Score,Right Thigh Circumference,Segmental ECW LA,Segmental ECW LL,Segmental ECW RA,Segmental ECW RL,Segmental ECW TR,Segmental ICW LA,Segmental ICW LL,Segmental ICW RA,Segmental ICW RL,Segmental ICW TR,Segmental body water LA,Segmental body water LL,Segmental body water RA,Segmental body water RL,Segmental body water TR,Skeletal Muscle Mass,Soft Lean Mass,Standard Body Weight,Standard Fat Perc,Standard Muscle Mass Perc,Systolic Blood Pressure,TBW/FFM,Target Weight,Total Body Water,Total Body Water Perc,Trunk Fat Free Mass,Trunk Fat Mass,Trunk Fat Perc,Trunk Fat Perc Score,Trunk Muscle Mass,Trunk Muscle Mass Score,VFA (Visceral Fat Area),Visceral Fat Rating,Waist Circumference,Waist-Hip ratio,Weight,Weight Control
0,0015d65e3205deb6bb6a8f0d57cc48547918f0f2,10,1.0,0.0,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.0,NaN
1,0015d65e3205deb6bb6a8f0d57cc48547918f0f2,18,1.0,0.0,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.0,NaN
2,0015d65e3205deb6bb6a8f0d57cc48547918f0f2,23,1.0,0.0,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.0,NaN
3,0015d65e3205deb6bb6a8f0d57cc48547918f0

In [9]:
# store to pickle file
mean_values.to_pickle('../data_processed/biometrics_mean_per_week.pkl')

In [40]:
mean_values = pd.read_pickle('../data_processed/biometrics_mean_per_week.pkl')
print(mean_values.shape)
all_users = mean_values.copy()

(90425, 122)


In [41]:
all_users.head(2)

,CloudId,MeasuredOnWeek,gender_m,gender_f,Age,BFM Control,BMI,Basal Metabolic Rate,Basal Metabolic Rate Score,Body cell mass,Bone Mass,Bone Mineral Content,Chest Circumference,Degree Of Obesity Perc,Diastolic Blood Pressure,Dry Lean Mass,ECW/TBW,ECW/TBW-LA,ECW/TBW-LLL,ECW/TBW-RA,ECW/TBW-RL,ECW/TBW-TR,Extra Cellular Water,Extra Cellular Water Perc,FFM Control,Fat Free Mass,Fat Free Mass Perc of Ideal Left Arm,Fat Free Mass Perc of Ideal Left Leg,Fat Free Mass Perc of Ideal Right Arm,Fat Free Mass Perc of Ideal Right Leg,Fat Free Mass Perc of Ideal Trunk,Fat Mass,Fat Mass Perc of Ideal Left Arm,Fat Mass Perc of Ideal Left Leg,Fat Mass Perc of Ideal Right Arm,Fat Mass Perc of Ideal Right Leg,Fat Mass Perc of Ideal Trunk,Fat mass Perc,Growth Score,HR At Rest,Height,Hip Circumference,InBody Score,Intra Cellular Water,Left Arm Circumference,Left Arm Fat Free Mass,Left Arm Fat Mass,Left Arm Fat Perc,Left Arm Fat Perc Score,Left Arm Muscle Circumference,Left Arm Muscle Mass,Left Arm Muscle Mass Score,Left Leg Fat Free Mass,Left Leg Fat Mass,Left Leg Fat Perc,Left Leg Fat Perc Score,Left Leg Muscle Mass,Left Leg Muscle Mass Score,Left Thigh Circumference,Leg Muscle Score,Metabolic Age,Minerals,Muscle Mass,Muscle Mass Balance Arm,Muscle Mass Balance Leg,Muscle Score,Neck Circumference,Obesity Degree,Obesity Degree of a Child,Power Threshold,Protein,Right Arm Circumference,Right Arm Fat Free Mass,Right Arm Fat Mass,Right Arm Fat Perc,Right Arm Fat Perc Score,Right Arm Muscle Mass,Right Arm Muscle Mass Score,Right Leg Fat Free Mass,Right Leg Fat Mass,Right Leg Fat Perc,Right Leg Fat Perc Score,Right Leg Muscle Mass,Right Leg Muscle Mass Score,Right Thigh Circumference,Segmental ECW LA,Segmental ECW LL,Segmental ECW RA,Segmental ECW RL,Segmental ECW TR,Segmental ICW LA,Segmental ICW LL,Segmental ICW RA,Segmental ICW RL,Segmental ICW TR,Segmental body water LA,Segmental body water LL,Segmental body water RA,Segmental body water RL,Segmental body water TR,Skeletal Muscle Mass,Soft Lean Mass,Standard Body Weight,Standard Fat Perc,Standard Muscle Mass Perc,Systolic Blood Pressure,TBW/FFM,Target Weight,Total Body Water,Total Body Water Perc,Trunk Fat Free Mass,Trunk Fat Mass,Trunk Fat Perc,Trunk Fat Perc Score,Trunk Muscle Mass,Trunk Muscle Mass Score,VFA (Visceral Fat Area),Visceral Fat Rating,Waist Circumference,Waist-Hip ratio,Weight,Weight Control
0,0015d65e3205deb6bb6a8f0d57cc48547918f0f2,10,1.0,0.0,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.0,NaN
1,0015d65e3205deb6bb6a8f0d57cc48547918f0f2,18,1.0,0.0,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.0,NaN


### Keep only columns where we have more than x% of values

## KNN


Fill the missing values with the mean of the values of the most similar users

In [42]:
# drop columns which contain NaN values for 10% of the rows
all_users.dropna(axis=1, thresh=int(0.5*len(all_users)), inplace=True)
all_users.shape

(90425, 48)

In [44]:
imputer = KNNImputer(n_neighbors=10)
imputed_data = imputer.fit_transform(all_users.drop(columns=['MeasuredOnWeek', 'CloudId']))

In [46]:
# store the imputed data in a new DataFrame
imputed_df = pd.DataFrame(imputed_data, columns=all_users.drop(columns=['MeasuredOnWeek', 'CloudId']).columns)
print(imputed_df.shape)
imputed_df.head(2)

(90425, 46)


,gender_m,gender_f,Age,BMI,Basal Metabolic Rate,Basal Metabolic Rate Score,Bone Mass,Degree Of Obesity Perc,Extra Cellular Water,Fat Free Mass,Fat Mass,Fat mass Perc,Height,Intra Cellular Water,Left Arm Fat Free Mass,Left Arm Fat Mass,Left Arm Fat Perc,Left Arm Muscle Mass,Left Leg Fat Free Mass,Left Leg Fat Mass,Left Leg Fat Perc,Left Leg Muscle Mass,Leg Muscle Score,Metabolic Age,Muscle Mass,Right Arm Fat Free Mass,Right Arm Fat Mass,Right Arm Fat Perc,Right Arm Muscle Mass,Right Leg Fat Free Mass,Right Leg Fat Mass,Right Leg Fat Perc,Right Leg Muscle Mass,Standard Body Weight,Standard Fat Perc,Standard Muscle Mass Perc,Total Body Water,Total Body Water Perc,Trunk Fat Free Mass,Trunk Fat Mass,Trunk Fat Perc,Trunk Fat Perc Score,Trunk Muscle Mass,Trunk Muscle Mass Score,Visceral Fat Rating,Weight
0,1.0,0.0,56.0,29.19,1938.7,9.5,3.62,23.93,18.81,67.98,20.17,22.88,174.0,30.12,4.114,1.13,17.4,4.09,10.749,3.04,20.69,10.5,85.4,46.0,63.19,4.189,1.1,16.6,4.07,10.897,3.06,20.7,10.59,71.17,17.0,56.08,48.93,56.47,35.161,11.33,21.69,-1.7,36.98,2.0,10.1,88.0
1,1.0,0.0,56.0,29.19,1938.7,9.5,3.62,23.93,18.81,67.98,20.17,22.88,174.0,30.12,4.114,1.13,17.4,4.09,10.749,3.04,20.69,10.5,85.4,46.0,63.19,4.189,1.1,16.6,4.07,10.897,3.06,20.7,10.59,71.17,17.0,56.08,48.93,56.47,35.161,11.33,21.69,-1.7,36.98,2.0,10.1,88.0


In [1]:
imputed_df.columns

NameError: name 'imputed_df' is not defined

In [47]:
imputed_df.to_pickle('../data_processed/biometrics_m50_imputed_knn.pkl')

In [48]:
# store the imputed data to a pickle file
imputed_df.to_pickle('../data_processed/biometrics_m50_imputed.pkl')

In [52]:
# read pickle file
imputed_df = pd.read_pickle('../data_processed/biometrics_m50_imputed.pkl')
imputed_df = pd.DataFrame(imputed_data, columns=all_users.drop(columns=['MeasuredOnWeek', 'CloudId']).columns)
print(imputed_df.shape)
print(imputed_df.isnull().sum())

(90425, 46)
gender_m                      0
gender_f                      0
Age                           0
BMI                           0
Basal Metabolic Rate          0
Basal Metabolic Rate Score    0
Bone Mass                     0
Degree Of Obesity Perc        0
Extra Cellular Water          0
Fat Free Mass                 0
Fat Mass                      0
Fat mass Perc                 0
Height                        0
Intra Cellular Water          0
Left Arm Fat Free Mass        0
Left Arm Fat Mass             0
Left Arm Fat Perc             0
Left Arm Muscle Mass          0
Left Leg Fat Free Mass        0
Left Leg Fat Mass             0
Left Leg Fat Perc             0
Left Leg Muscle Mass          0
Leg Muscle Score              0
Metabolic Age                 0
Muscle Mass                   0
Right Arm Fat Free Mass       0
Right Arm Fat Mass            0
Right Arm Fat Perc            0
Right Arm Muscle Mass         0
Right Leg Fat Free Mass       0
Right Leg Fat Mass          

In [55]:
# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(imputed_df)
pd.DataFrame(scaled_data, columns=imputed_df.columns).describe()

,gender_m,gender_f,Age,BMI,Basal Metabolic Rate,Basal Metabolic Rate Score,Bone Mass,Degree Of Obesity Perc,Extra Cellular Water,Fat Free Mass,Fat Mass,Fat mass Perc,Height,Intra Cellular Water,Left Arm Fat Free Mass,Left Arm Fat Mass,Left Arm Fat Perc,Left Arm Muscle Mass,Left Leg Fat Free Mass,Left Leg Fat Mass,Left Leg Fat Perc,Left Leg Muscle Mass,Leg Muscle Score,Metabolic Age,Muscle Mass,Right Arm Fat Free Mass,Right Arm Fat Mass,Right Arm Fat Perc,Right Arm Muscle Mass,Right Leg Fat Free Mass,Right Leg Fat Mass,Right Leg Fat Perc,Right Leg Muscle Mass,Standard Body Weight,Standard Fat Perc,Standard Muscle Mass Perc,Total Body Water,Total Body Water Perc,Trunk Fat Free Mass,Trunk Fat Mass,Trunk Fat Perc,Trunk Fat Perc Score,Trunk Muscle Mass,Trunk Muscle Mass Score,Visceral Fat Rating,Weight
count,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04,9.042500e+04
mean,-1.404977e-16,1.404977e-16,-1.253321e-16,-2.571076e-16,-6.963594e-16,2.152255e-16,-1.101548e-15,2.599365e-16,5.752705e-16,1.728719e-16,1.304397e-16,-1.092236e-17,-2.115638e-15,-5.397532e-16,-2.344771e-16,-1.219533e-16,2.007671e-17,-3.413434e-16,-7.458636e-16,1.310683e-16,7.165540e-16,3.671170e-16,-6.700750e-17,-3.815754e-16,2.841385e-16,-1.360973e-16,-7.009169e-17,-2.884603e-16,-3.064547e-17,6.902303e-16,-2.923106e-17,1.822227e-16,-1.077149e-15,-2.170328e-16,1.881160e-16,-6.679141e-17,7.727373e-16,-9.588103e-16,7.151396e-16,4.230646e-16,-2.806811e-16,-1.552704e-16,-8.769319e-17,-4.337513e-17,5.720488e-17,1.643854e-16
std,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00
min,-1.214924e+00,-8.230967e-01,-2.411656e+00,-6.107220e+00,-2.244213e+00,-3.032205e+00,-3.986481e+00,-2.334413e+00,-2.329027e+00,-2.364726e+00,-1.899771e+00,-2.563399e+00,-1.003757e+01,-1.967503e+00,-1.956453e+00,-1.430087e+00,-2.240567e+00,-1.993352e+00,-2.826006e+00,-2.063920e+00,-2.473147e+00,-2.240323e+00,-3.801997e+00,-2.008317e+00,-3.561400e+00,-2.093363e+00,-1.439468e+00,-2.185840e+00,-2.041760e+00,-2.821260e+00,-2.041666e+00,-2.444165e+00,-2.152230e+00,-2.911189e+00,-1.294842e+00,-2.151000e+00,-2.304076e+00,-8.347740e+00,-2.971606e+00,-1.961751e+00,-2.472051e+00,-2.152407e+00,-2.660280e+00,-4.231314e+00,-1.599506e+00,-2.801733e+00
25%,-1.214924e+00,-8.230967e-01,-7.556577e-01,-6.670063e-01,-7.705706e-01,-6.134957e-01,-7.721879e-01,-6.319198e-01,-8.249240e-01,-8.167634e-01,-6.652687e-01,-7.215919e-01,-7.367228e-01,-8.625051e-01,-8.538843e-01,-5.343404e-01,-7.129614e-01,-8.084371e-01,-8.694025e-01,-6.899076e-01,-7.984082e-01,-8.911056e-01,-6.926235e-01,-7.101350e-01,-8.134509e-01,-8.665997e-01,-6.135529e-01,-7.052603e-01,-8.259541e-01,-8.530242e-01,-6.923987e-01,-8.122620e-01,-9.143011e-01,-7.220904e-01,-7.941926e-01,-9.081997e-01,-8.563061e-01,-6.534716e-01,-7.132481e-01,-6.997549e-01,-6.639220e-01,-7.440961e-01,-7.420556e-01,-6.232040e-01,-6.854359e-01,-6.984633e-01
50%,8.230967e-01,-8.230967e-01,3.922167e-02,-1.614996e-01,-

In [56]:
# Apply PCA to reduce dimensionality to 2 components
pca = PCA(n_components=2)
pca_result = pca.fit_transform(scaled_data)

In [59]:
# Step 3: Create a DataFrame for the PCA result
pca_df = pd.DataFrame(pca_result, columns=['PCA1', 'PCA2'])
pca_df['MeasuredOnWeek'] = all_users['MeasuredOnWeek'].values
pca_df['CloudId'] = all_users['CloudId'].values
pca_df.head()

,PCA1,PCA2,MeasuredOnWeek,CloudId
0,4.503959,0.922746,10,0015d65e3205deb6bb6a8f0d57cc48547918f0f2
1,4.503959,0.922746,18,0015d65e3205deb6bb6a8f0d57cc48547918f0f2
2,4.503959,0.922746,23,0015d65e3205deb6bb6a8f0d57cc48547918f0f2
3,4.276904,0.936682,24,0015d65e3205deb6bb6a8f0d57cc48547918f0f2
4,4.206436,0.645258,25,0015d65e3205deb6bb6a8f0d57cc48547918f0f2


### Do it for multiple users

In [60]:
#merge on fields MeasuredOnWeek and CloudId
all = pd.merge(pca_df, all_users, on=['MeasuredOnWeek', 'CloudId'])
all.head()

,PCA1,PCA2,MeasuredOnWeek,CloudId,gender_m,gender_f,Age,BMI,Basal Metabolic Rate,Basal Metabolic Rate Score,Bone Mass,Degree Of Obesity Perc,Extra Cellular Water,Fat Free Mass,Fat Mass,Fat mass Perc,Height,Intra Cellular Water,Left Arm Fat Free Mass,Left Arm Fat Mass,Left Arm Fat Perc,Left Arm Muscle Mass,Left Leg Fat Free Mass,Left Leg Fat Mass,Left Leg Fat Perc,Left Leg Muscle Mass,Leg Muscle Score,Metabolic Age,Muscle Mass,Right Arm Fat Free Mass,Right Arm Fat Mass,Right Arm Fat Perc,Right Arm Muscle Mass,Right Leg Fat Free Mass,Right Leg Fat Mass,Right Leg Fat Perc,Right Leg Muscle Mass,Standard Body Weight,Standard Fat Perc,Standard Muscle Mass Perc,Total Body Water,Total Body Water Perc,Trunk Fat Free Mass,Trunk Fat Mass,Trunk Fat Perc,Trunk Fat Perc Score,Trunk Muscle Mass,Trunk Muscle Mass Score,Visceral Fat Rating,Weight
0,4.503959,0.922746,10,0015d65e3205deb6bb6a8f0d57cc48547918f0f2,1.0,0.0,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.0
1,4.503959,0.922746,18,0015d65e3205deb6bb6a8f0d57cc48547918f0f2,1.0,0.0,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.0
2,4.503959,0.922746,23,0015d65e3205deb6bb6a8f0d57cc48547918f0f2,1.0,0.0,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.0
3,4.276904,0.936682,24,0015d65e3205deb6bb6a8f0d57cc48547918f0f2,1.0,0.0,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87.0
4,4.206436,0.645258,25,0015d65e3205deb6bb6a8f0d57cc48547918f0f2,1.0,0.0,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.0


In [61]:
unique_cloud_ids = all['CloudId'].unique()[:30]

# Initialize a new figure
fig = go.Figure()

# Iterate over each selected CloudId and add a trace for each
for cloud_id in unique_cloud_ids:
    user_data = all[all['CloudId'] == cloud_id]
    fig.add_trace(go.Scatter(
        x=user_data['PCA1'],
        y=user_data['PCA2'],
        mode='lines+markers',
        line_shape='spline',
        name=f'{cloud_id[:4]}',
        text=user_data.apply(lambda row: '<br>'.join([f'{col}: {row[col]}' for col in ['Weight', 'Basal Metabolic Rate', 'Basal Metabolic Rate Score', 'Degree Of Obesity Perc']]), axis=1),
        hoverinfo='text',
        marker=dict(size=2, showscale=True),  # Adjusted marker size
        line=dict(shape='spline')
    ))

fig.update_layout(
    title='2D PCA Plot of User Data by Week with Temporal Evolution',
    xaxis_title='PCA1',
    yaxis_title='PCA2',
    showlegend=True
)

fig.show()